In [11]:
import numpy as np
import torch
from torch.nn import functional as F
import transformers
import pdb
from datasets import load_dataset
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW
from transformers import GPT2LMHeadModel
from transformers import GPT2TokenizerFast
from transformers import Trainer
from transformers import TrainingArguments
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [27]:
"""
Trying a different dataset
"""

def get_dataset():
    train_dataset = load_dataset("huggingartists/taylor-swift", split="train")
    eval_dataset = load_dataset("huggingartists/taylor-swift", split="train")
    return train_dataset, eval_dataset

In [24]:
def load_init_model_optimizer_tokenizer():
    pretrained_weights = 'google-t5/t5-small'
    model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_weights)
    tokenizer = AutoTokenizer.from_pretrained(pretrained_weights)
    # models are usually loaded in eval() mode, so set this to train()
    model.train()
    # initialize the optimizer
    optimizer = AdamW(model.parameters(), lr=1e-5)
    # using weight decay
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)

    return model, optimizer, tokenizer

In [25]:
"""
in model.generate you need to set return_dict_in_generate=False
to return torch.LongTensor
"""
def sampling_loop(model, input_ids, attention_mask, num_decode_steps=10):
    output = model.generate(input_ids, attention_mask=attention_mask, max_new_tokens=num_decode_steps)

In [28]:
train_dataset, test_dataset = get_dataset()
print(train_dataset)

Dataset({
    features: ['text'],
    num_rows: 762
})


In [38]:
def tokenize(batch):
    data = tokenizer(batch['text'], padding=True, truncation=True)
    return data

train_dataset, test_dataset = get_dataset()
model, optimizer, tokenizer = load_init_model_optimizer_tokenizer()
tokenizer.pad_token = tokenizer.eos_token
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset= test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask'])

In [31]:
print(len(train_dataset))

762


In [36]:
print(train_dataset)

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 762
})


In [32]:
print(model.config)

T5Config {
  "_name_or_path": "google-t5/t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 

In [33]:
"""
DOING
"""

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [34]:
import warnings;
warnings.filterwarnings('ignore');


training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset
    tokenizer=tokenizer,                 # tokenizer
)

In [35]:
trainer.train()

[2024-09-04 14:12:15,047] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to mps (auto detect)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[2024-09-04 14:12:15,829] torch.distributed.elastic.multiprocessing.redirects: [WARNING] NOTE: Redirects are currently not supported in Windows or MacOs.


ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds